#### Import libraries

In [1]:
import os
import keras
import joblib
import pickle
import warnings
import numpy as np
import pandas as pd
from keras.layers import LSTM
from keras.layers import Dense
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

np.set_printoptions(suppress=True)
plt.rcParams["figure.figsize"] = (20,10)
warnings.filterwarnings('ignore')

#### Upload the data

In [2]:
merged_df = pickle.load(open("data/processed/merged_data.p", "rb"))
merged_df.head()

,Code,Région,TMin (°C),TMax (°C),TMoy (°C),Quarter,Weekday,Holiday,HDD,CDD,Electric Consumption (MW),seasons
Date,,,,,,,,,,,,
2016-01-01,11,Île-de-France,3.30,8.76,6.03,1,4,1,9.47,0.0,388905.0,Winter
2016-01-02,11,Île-de-France,6.60,10.86,8.73,1,5,1,6.77,0.0,455571.0,Winter
2016-01-03,11,Île-de-France,5.30,8.08,6.69,1,6,1,8.81,0.0,534162.0,Winter
2016-01-04,11,Île-de-France,5.46,9.76,7.61,1,0,0,7.89,0.0,466203.0,Winter
2016-01-05,11,Île-de-France,5.12,9.00,7.06,1,1,0,8.44,0.0,359755.0,Winter


#### Modelling and Evaluation

<div> 
    <ul>
      <li>Convert the data to supervised data.</li>
      <li>Normalize the data.</li>
      <li>Structure the neural network.</li>
      <li>Train the network.</li>
      <li>Evaluate the network.</li>
      <li>Store the weights of the neural network.</li>
    </ul>     
</div>

In [3]:
rgional_df = {}
normed_df = {}
suprevised_df = {}
norm_dict = {}

regions = merged_df.groupby(["Code"])

In [4]:
for code in regions.groups:
    rgional_df = regions.get_group(code)[["TMoy (°C)","HDD","CDD","Electric Consumption (MW)"]]
    rgional_df.columns = ['temp','HDD','CDD','elect_demand']

    suprevised_df = rgional_df.copy()
    suprevised_df.loc["2019-03-12":"2019-03-31"] = suprevised_df.loc["2018-03-12":"2018-03-31"]

    for obs in range(1,30):
        suprevised_df["T_" + str(obs)] = suprevised_df["elect_demand"].shift(-obs)

    suprevised_df.dropna(inplace=True)
    last = len(suprevised_df.columns.values)
    suprevised_df.insert(last-1, 'elect_demand', suprevised_df.pop('elect_demand'))

    train = suprevised_df.loc["2016":"2018"]
    test = suprevised_df.loc["2019"]

    scaler = MinMaxScaler(feature_range=(0, 1))
    # split into input and outputs
    scaler = scaler.fit(train)

    train = scaler.transform(train)
    test = scaler.transform(test)

    train_X, train_y = train[:,:-1], train[:,-1]
    test_X, test_y =  test[:,:-1], test[:,-1]

    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

    # create and fit the LSTM network
    model = Sequential() # New Instance of Model Object
    model.add(LSTM(50, return_sequences=True, input_shape=(1, train_X.shape[2])))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer='ADAM')

    hist =model.fit(train_X, train_y, epochs=300, shuffle=False,batch_size=64, validation_data=(test_X, test_y), 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=20)], verbose=0)

    yhat = model.predict(test_X)
    predicted = yhat.flatten() * (scaler.data_max_[-1] - scaler.data_min_[-1]) + scaler.data_min_[-1]
    actual = test_y * (scaler.data_max_[-1] - scaler.data_min_[-1]) + scaler.data_min_[-1]

    print("MSE : {:.3f}".format(mean_squared_error(actual, predicted,squared=False)))

    os.makedirs(f'data/forecasting/{code}', exist_ok=True)
    model.save(f'data/forecasting/{code}/lstm.h5');
    joblib.dump(scaler, f'data/forecasting/{code}/scaler');
    
    print(f"Forecasting for region {code} has finished successfully!")

MSE : 41652.558
Forecasting for region 11 has finished successfully!
MSE : 11981.454
Forecasting for region 24 has finished successfully!
MSE : 14573.561
Forecasting for region 27 has finished successfully!
MSE : 13915.021
Forecasting for region 28 has finished successfully!
MSE : 23288.052
Forecasting for region 32 has finished successfully!
MSE : 23355.476
Forecasting for region 44 has finished successfully!
MSE : 18575.600
Forecasting for region 52 has finished successfully!
MSE : 15398.254
Forecasting for region 53 has finished successfully!
MSE : 25354.036
Forecasting for region 75 has finished successfully!
MSE : 26268.492
Forecasting for region 76 has finished successfully!
MSE : 39208.176
Forecasting for region 84 has finished successfully!
MSE : 21214.792
Forecasting for region 93 has finished successfully!
